In [39]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from random import randint
from time import sleep
import re

secrets_file = open("spotkey.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [40]:
url = "https://audiohype.io/resources/the-best-spotify-playlists/"

response = requests.get(url)
response.status_code

200

In [41]:
soup = BeautifulSoup(response.content, "html.parser")
data = soup.select('.rank-math-link')

In [42]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

def reg_replace(x):
    return re.findall(r'\bhttps?:\/\/open.spotify\.com\/playlist\/([^\s?]+)',x)

In [43]:
name = [x.get_text() for x in data]
uri = flatten([reg_replace(x['href']) for x in data])

playlist_df = pd.DataFrame({"name": name[:-1],
                       "uri": uri
                      })

playlist_df.loc[len(playlist_df.index)] = ['Really big one', '4rnleEAOdmFAbRcNCgZMpY']

In [47]:
def playlist_dff(playlist_id):
    flag=False
    
    #step1
    playlist_lst = ['track_id',"artist","track_name","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_lst)
    
    try:
        #step2
        playlist = sp.playlist_items(playlist_id)
        while playlist['next']!=None:
            
            if flag:
                playlist = sp.next(playlist)
                # respectful nap
                wait_time = random.randint(1,3)
                print('Napping for ',wait_time,'s')
                sleep(wait_time)
            
            for track in playlist["items"]:
                
                # Create empty dict
                playlist_features = {}
                # Get metadata
                playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
                playlist_features["track_name"] = track["track"]["name"]
                playlist_features["track_id"] = track["track"]["id"]

                # Get audio features
                audio_features = sp.audio_features(track["track"]["id"])[0]
                for feature in playlist_lst[3:]:
                    playlist_features[feature] = audio_features[feature]

                # Concat the dfs
                track_df = pd.DataFrame(playlist_features, index = [0])
                playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
            
            flag = True
            print(playlist_id, 'Success!')        
    except:
        print(playlist_id, 'Invalid playlist')
        
    return playlist_df

In [50]:
spotify_df = pd.DataFrame()

for i in playlist_df['uri']:
    spotify_df = pd.concat([spotify_df, playlist_dff(i)], axis = 0)


31ymdYCITDnZRtkKzP3Itp Success!
Napping for  1 s
31ymdYCITDnZRtkKzP3Itp Success!
37i9dQZF1DXa71eg5j9dKZ Success!
Napping for  1 s
37i9dQZF1DXa71eg5j9dKZ Success!
5wtqmpRl17iVz2nW8U6njL Invalid playlist
0NCspsyf0OS4BsPgGhkQXM Success!
Napping for  2 s
0NCspsyf0OS4BsPgGhkQXM Success!
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  3 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  1 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  3 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  2 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  1 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  2 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  3 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  1 s
19y0UVk0bcrJWEqMwBHosj Success!
Napping for  1 s
19y0UVk0bcrJWEqMwBHosj Success!
4oOZJEq1TBUti6PSouTo5M Success!
Napping for  2 s
4oOZJEq1TBUti6PSouTo5M Success!
068WHS0zOWsqvn2uIBYb5D Success!
Napping for  2 s
068WHS0zOWsqvn2uIBYb5D Success!
0MSCX9tZWQmitMQsfhvZIl Success!
Napping for  2 s
0MSCX9tZWQmitMQsfhvZIl Success!
1

In [53]:
spotify_df.drop_duplicates().to_csv('spotify_df.csv',index=False)